# Método de Ondas Planas 2D

Programa para el cálculo de estructura de bandas en cristales fotónicos 2D

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#normalización
a = 1e-6

r = 0.4*a              #radio
eps1 = 9               #Permitividad del material 1
eps2 = 1               #Permitividad del material 2
precis = 5             #Número de k-points entre puntos de alta simetría
nG = 4                 #Número de ondas planas
precisStruct = 30      #Número de nodos de discretización al interior de la celda unitaria


In [12]:
#Función dieléctrica para los nodos de la malla
x = np.arange(-a/2,a/2+a/precisStruct,a/precisStruct)
y = np.arange(-a/2,a/2+a/precisStruct,a/precisStruct)

xset = list()
yset = list()
struct = np.zeros((len(x),len(x)))

for i in range(len(x)):
    for j in range(len(y)):
        if(np.sqrt(x[i]**2 + y[j]**2) < r):
            struct[i,j] = 1/eps2
            xset.append(i)
            yset.append(j)
        else:
            struct[i,j] = 1/eps1
            xset.append(i)
            yset.append(j) 
        

In [14]:
dS = (a/precisStruct)**2

xMesh = np.meshgrid(xset[0:len(xset)-1])
yMesh = np.meshgrid(yset[0:len(yset)-1])

In [17]:
structMesh = struct[0:len(xset)-1,0:len(yset)-1]*dS/(max(xset) - min(xset))**2

SyntaxError: invalid syntax (1769260183.py, line 2)